In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import TextVectorization, Embedding, SimpleRNN, Dense, LSTM, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
import gensim.downloader as gensim_downloader
import gensim
import multiprocessing
from mlflow import MlflowClient
import mlflow
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import optuna

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix 
from nltk.corpus import stopwords  
from nltk.tokenize import TweetTokenizer, WordPunctTokenizer, RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer, LancasterStemmer, SnowballStemmer
from tqdm import tqdm
tqdm.pandas()


import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


import sys
from pathlib import Path
cwd = Path.cwd()
parent = cwd.parent
sys.path.append(str(parent))


from Source.preprocess_data import *  ## import all functions from preprocess_data.py
from Source.postprocess_data import * ## import all functions from postprocess_data.py
from Source.utils import *  ## import all functions from utils.py
import nltk


client = MlflowClient(tracking_uri="http://localhost:8080")

mlruns_path = Path("../mlruns").resolve() 
mlflow_uri = mlruns_path.as_uri()
mlflow.set_tracking_uri(mlflow_uri)

nw = multiprocessing.cpu_count()



os.environ["TF_KERAS"]='1'
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("GPUs disponibles :", tf.config.list_physical_devices("GPU"))
print("Version TF :", tf.__version__)

c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


2.10.1
Num GPUs Available:  1
GPUs disponibles : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Version TF : 2.10.1


In [2]:
df = pd.read_csv('https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip',
                header=None,
                compression='zip',
                encoding='cp1252')

df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
data_size = 0.02
sample_df, _ = train_test_split(df, test_size=1-data_size, random_state=42, stratify=df['target'])
sample_df = sample_df.reset_index(drop=True)
print(f"Sample size: {sample_df.shape[0]} rows")
data_numrows = sample_df.shape[0]
# On ne garde que les colonnes 'target' et 'text'
sample_df = sample_df[['target', 'text']]
sample_df["target"] = sample_df["target"].apply(lambda x: 0 if x == 0 else 1)
sample_df.to_csv('../Data/raw_data.csv', index=False)
dataset = mlflow.data.from_pandas(
    sample_df,
    source="../Data/raw_data.csv",
    name="dataset_v1"
)


Sample size: 32000 rows


c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\data\dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../Data/raw_data.csv'. Exception: 
  return _dataset_source_registry.resolve(
c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\data\dataset_source_registry.py:148: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(


# Séparation train/validation

In [3]:
# Data
X_raw = sample_df['text']
y = sample_df['target']
X_train, X_val, y_train, y_val = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)


In [4]:
type(X_val)

pandas.core.series.Series

# Préparation de l'experience de base (baseline)


## Pré-traitement des dataframes

In [5]:
num_words = 20000
max_len = 10
min_count = 3

X_sentence_train, tokenizer, sentences_train = preprocess_data_embedding(X_raw=X_train, 
                                                        stem_lem_func=PorterStemmer().stem,
                                                        tokenizer=None, 
                                                        stop_words=stopwords.words('english'), 
                                                        min_count=min_count,
                                                        max_len = max_len, 
                                                        num_words=num_words, 
                                                        return_sentences=True) 
X_sentence_val = preprocess_data_embedding(X_raw=X_val, 
                                                        stem_lem_func=PorterStemmer().stem,
                                                        tokenizer=tokenizer, 
                                                        stop_words=stopwords.words('english'), 
                                                        min_count=1, # mincount = 1 car on est sur le jeu de validation
                                                        max_len = max_len, 
                                                        num_words=num_words) 

# Création d'un embedding de base custom pour notre modèle


In [6]:
from gensim.models import Word2Vec
latent_dim = 50
print("Build & train Word2Vec model ...")

w2v_model = Word2Vec(
    sentences=sentences_train, 
    vector_size=latent_dim,  # dimension de l’espace latent
    window=5,         # taille du contexte
    min_count=min_count,      # ignorer les mots trop rares
    workers=4,        # parallélisme CPU
    sg=0,              # 1 = skip-gram, 0 = CBOW
    epochs=100
)


model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

found = sum(1 for w in tokenizer.word_index if w in w2v_model.wv)
coverage = found / len(tokenizer.word_index)
print(f"Coverage: {coverage*100:.2f}%")

vectors = np.array([w2v_model.wv[w] for w in tokenizer.word_index if w in w2v_model.wv])
print("Mean norm:", np.mean(np.linalg.norm(vectors, axis=1)))


Build & train Word2Vec model ...
Vocabulary size: 5491
Word2Vec trained
Coverage: 99.98%
Mean norm: 6.3330092


## Création de la matrice d'embedding 

In [7]:
embedding_matrix, vocab_size = build_embedding_matrix(tokenizer=tokenizer,
                                          embedding_model=model_vectors, 
                                          latent_dim=latent_dim
                                          )

Embedding matrix shape: (5493, 50)
Words found in pretrained embeddings: 5491/5493 (99.96%)


## Création du modèle simple avec RNN

In [8]:
model =  build_base_RNN(vocab_size=vocab_size, 
                        latent_dim=latent_dim,
                        input_length=max_len, 
                        embedding_matrix=embedding_matrix,
                        rnn_size = 64)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 50)            274650    
                                                                 
 simple_rnn (SimpleRNN)      (None, 10, 64)            7360      
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 282,075
Trainable params: 7,425
Non-trainable params: 274,650
_________________________________________________________________


## Callbacks pour l'entrainement

In [9]:



checkpoint = ModelCheckpoint("./Models/baselineRNN.h5", monitor='val_loss', verbose=0, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, min_lr=1e-5)
optimizer = Adam(learning_rate=1e-3)

callbacks_list = [checkpoint, es, lr_scheduler]
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

## Entrainement

In [10]:
with tf.device("/GPU:0"):
    history = model.fit(X_sentence_train, y_train, epochs=50, batch_size=64, validation_data=(X_sentence_val,y_val), callbacks=callbacks_list, verbose=0)


## Post-traitement

In [11]:
y_pred_proba = model.predict(X_sentence_val)
y_pred = (y_pred_proba>0.5)


output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

output_dict

200/200 [==============================] - 1s 4ms/step


{'Accuracy': 0.70421875,
 'F1_negatif': 0.7044496487119438,
 'F1_positif': 0.7039874902267397,
 'Recall_negatif': 0.705,
 'Recall_positif': 0.7034375,
 'Precision_negatif': 0.7039001560062402,
 'Precision_positif': 0.7045383411580595,
 'ROC_AUC': 0.777373828125}

## Liste des hyper-paramètres à optimiser 


- Prétraitement : 
    - Stemming ou Lemmatisation
    - Taille du vocabulaire (num_words)
    - Nombre minimum d'occurences (min_count)

- Embedding : 
    - Word2Vec/FastText/Glove (préentrainés)
    - Word2Vec/FastText (customisés)
    - Dimension latente de l'embedding
- Modèle : 
    - Couche SimpleRNN ou LSTM
    - Dimension de la couche d'entrainement
    - Fine-tuning ou non des embeddings ? 

# Experimentations sur les modèles de RNN classiques 

## Préparation

In [12]:

Stemmer_dict = {'WordNetLemmatizer': WordNetLemmatizer().lemmatize, 
                'PorterStemmer': PorterStemmer().stem, 
                'LancasterStemmer':LancasterStemmer().stem, 
                'SnowballStemmer' : SnowballStemmer("english").stem
}

list(Stemmer_dict.keys())

['WordNetLemmatizer', 'PorterStemmer', 'LancasterStemmer', 'SnowballStemmer']

## Core fonction

In [13]:
#Fonction à optimiser pour optuna

def embedding_eva_pre(trial):
    # Hyperparamètres
    ## Prétraitement
    min_count = trial.suggest_int('min_count',1,10)
    num_words = trial.suggest_int('num_words',5000,50000)
    max_len   = trial.suggest_int('max_len',2,30)
    stemmer   = trial.suggest_categorical('stemmer',list(Stemmer_dict.keys()))
    ## Embedding
    latent_dim = 50
    ## Modèle
    rnn_size = 64
    ## Entrainement
    epochs = 50
    lr = 1e-3
    ## Savepath des poids du modèle
    model_savepath = "./Models/baselineRNN_pre.h5"



    with mlflow.start_run(nested=True):
        mlflow.log_input(dataset)

        mlflow.log_params(params={
            'num_words':num_words,               
            'max_len': max_len,
            'min_count': min_count,
            'stemmer': stemmer, 
            'latent_dim': latent_dim, 
            'rnn_size': rnn_size, 
            'epochs': epochs, 
            'learning_rate': lr,
            "data_size": data_size,
            "data_numrows": data_numrows, 
        })

        # Prétraitement
        X_sentence_train, tokenizer, sentences_train = preprocess_data_embedding(X_raw=X_train, 
                                                        stem_lem_func=Stemmer_dict[stemmer],
                                                        tokenizer=None, 
                                                        stop_words=stopwords.words('english'), 
                                                        min_count=min_count,
                                                        max_len = max_len, 
                                                        num_words=num_words, 
                                                        return_sentences=True) 
        X_sentence_val = preprocess_data_embedding(X_raw=X_val, 
                                                        stem_lem_func=PorterStemmer().stem,
                                                        tokenizer=tokenizer, 
                                                        stop_words=stopwords.words('english'), 
                                                        min_count=1, # mincount = 1 car on est sur le jeu de validation
                                                        max_len = max_len, 
                                                        num_words=num_words) 
        # Embedding(custom)
        
        w2v_model = Word2Vec(
            sentences=sentences_train, 
            vector_size=latent_dim,  # dimension de l’espace latent
            window=5,         # taille du contexte
            min_count=min_count,      # ignorer les mots trop rares
            workers=4,        # parallélisme CPU
            sg=0,              # 1 = skip-gram, 0 = CBOW
            epochs=50
            )


        model_vectors = w2v_model.wv
        w2v_words = model_vectors.index_to_key

        embedding_matrix, vocab_size = build_embedding_matrix(tokenizer=tokenizer,
                                          embedding_model=model_vectors, 
                                          latent_dim=latent_dim
                                          )

        
        # Modèle
        model =  build_base_RNN(vocab_size=vocab_size, 
                        latent_dim=latent_dim,
                        input_length=max_len, 
                        embedding_matrix=embedding_matrix,
                        rnn_size = rnn_size)
        ## Callbacks
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]
        ## Compilation
        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        #Entrainement
        with tf.device("/GPU:0"):
            history = model.fit(X_sentence_train, y_train, epochs=epochs, batch_size=64, validation_data=(X_sentence_val,y_val), callbacks=callbacks_list, verbose=0)

        model.load_weights(model_savepath)

        # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(X_sentence_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.sklearn.log_model(model, "model")
        acc = output_dict["Accuracy"]
    return acc

## Définition de l'experiment MLFlow/Optuna

In [14]:
# Création de l'étude Optuna et optimisation
print("Starting hyperparameter optimization with Optuna...")
print("Setting up MLflow experiment...")
mlflow.set_experiment("optuna_word_embedding_experiment_preprocessin")
exp_id = mlflow.get_experiment_by_name("optuna_word_embedding_experiment_preprocessin").experiment_id

experiment_description = (
    "Cette experience contient les différents tests pour le modèle RNN simple. "
    "Ici on évalue simplement l'impact des différents prétraitements sur un modèle avec simpleRNN"
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "simple-RNN-preprocessing",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Starting hyperparameter optimization with Optuna...
Setting up MLflow experiment...


Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

## Lancement de l'optimisation

In [15]:
# Lancement de l'optimisation avec Optuna
print("Starting optimization trials...")
with mlflow.start_run(run_name="optuna_word_embedding_experiment_preprocessin"):
    study = optuna.create_study(direction="maximize")
    study.optimize(embedding_eva_pre, n_trials=50)

    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_accuracy", study.best_value)

print("Optimization completed.")

2025/10/06 13:55:58 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmp98wsciu1\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.7.2', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/10/06 13:55:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-10-06 13:55:58,752] Trial 27 finished with value: 0.709375 and parameters: {'min_count': 4, 'num_words': 36201, 'max_len': 20, 'stemmer': 'PorterStemmer'}. Best is trial 15 with value: 0.7209375.


Embedding matrix shape: (3622, 50)
Words found in pretrained embeddings: 3620/3622 (99.94%)
200/200 [==============================] - 2s 9ms/step


2025/10/06 13:58:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


INFO:tensorflow:Assets written to: ram://881c32db-fdf1-4e2a-87cb-24dd87795d63/assets


INFO:tensorflow:Assets written to: ram://881c32db-fdf1-4e2a-87cb-24dd87795d63/assets
2025/10/06 13:58:43 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmp3pwqdvqx\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.7.2', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/10/06 13:58:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-10-06 13:58:43,259] Trial 28 finished with value: 0.71171875 and parameters: {'min_count': 5, 'num_words': 24064, 'max_len': 26, 'stemmer': 'PorterStemmer'}. Best is trial 15 with value: 0.7209375.


...

Embedding matrix shape: (15701, 50)
Words found in pretrained embeddings: 15699/15701 (99.99%)
200/200 [==============================] - 2s 8ms/step


2025/10/06 14:56:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


INFO:tensorflow:Assets written to: ram://91ba722e-3619-470f-a02d-9334bb029090/assets


INFO:tensorflow:Assets written to: ram://91ba722e-3619-470f-a02d-9334bb029090/assets
2025/10/06 14:56:14 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\bassm\AppData\Local\Temp\tmpa37964to\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.7.2', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/10/06 14:56:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-10-06 14:56:15,056] Trial 49 finished with value: 0.68671875 and parameters: {'min_count': 1, 'num_words': 20160, 'max_len': 27, 'stemmer': 'LancasterStemmer'}. Best is trial 45 with value: 0.72671875.


Optimization completed.


## Extraction meilleur modèle 

In [16]:
client = MlflowClient(tracking_uri="http://localhost:8080")
experiment_id = mlflow.get_experiment_by_name("optuna_word_embedding_experiment_preprocessin").experiment_id
runs = client.search_runs(experiment_id)

# Métrique pour sélectionner le meilleur modèle
metric_to_optimize = "Accuracy" # liste des métriques enregistrées dans postprocess_data.py ou sur l'UI MLflow
best_run = max(runs, key=lambda run: run.data.metrics.get(metric_to_optimize, float('-inf')))
print(f"Best run ID: {best_run.info.run_id} with metrics:")
for key, value in best_run.data.metrics.items():
    print(f"{key}: {value}")
print(f"Best run parameters:")
for key, value in best_run.data.params.items():
    print(f"{key}: {value}")

# Enregistrement du meilleur modèle
best_model_uri = f"runs:/{best_run.info.run_id}/model"
registered_model_name = "simple_rnn_best_pre"
registered_model = mlflow.register_model(best_model_uri, registered_model_name)
# Enregistrement des paramètres sous forme de tags dans le modèle enregistré
for key, value in best_run.data.params.items():
    print(f"Setting tag {key} = {value} in registered model")
    client.set_model_version_tag(
        name=registered_model_name,
        version=str(registered_model.version),
        key=str(key),
        value=str(value))

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Best run ID: ec4aa4d9ebd448c192219024711a0f0e with metrics:
Accuracy: 0.72671875
F1_negatif: 0.7226011102299762
F1_positif: 0.730715935334873
Precision_negatif: 0.7336553945249598
Precision_positif: 0.7201820940819423
Recall_negatif: 0.711875
Recall_positif: 0.7415625
ROC_AUC: 0.8004910644531249
Best run parameters:
data_numrows: 32000
data_size: 0.02
epochs: 50
latent_dim: 50
learning_rate: 0.001
max_len: 24
min_count: 1
num_words: 27395
rnn_size: 64
stemmer: PorterStemmer


Registered model 'simple_rnn_best_pre' already exists. Creating a new version of this model...
2025/10/06 14:56:19 WARNING mlflow.tracking._model_registry.fluent: Run with id ec4aa4d9ebd448c192219024711a0f0e has no artifacts at artifact path 'model', registering model based on models:/m-4710df3fd2de4a6e83444f1f96a00742 instead
Created version '3' of model 'simple_rnn_best_pre'.


Setting tag data_numrows = 32000 in registered model
Setting tag data_size = 0.02 in registered model
Setting tag epochs = 50 in registered model
Setting tag latent_dim = 50 in registered model
Setting tag learning_rate = 0.001 in registered model
Setting tag max_len = 24 in registered model
Setting tag min_count = 1 in registered model
Setting tag num_words = 27395 in registered model
Setting tag rnn_size = 64 in registered model
Setting tag stemmer = PorterStemmer in registered model


# Experimentation sur les embeddings (custom)


## Préparation

In [28]:
from gensim.models import FastText, Word2Vec

embedding_dict = {'Word2Vec':Word2Vec, 
                  'FastText':FastText}

## Core fonction

In [29]:
#Fonction à optimiser pour optuna

## Prétraitement
min_count = 2
num_words = 30000
max_len   = 30
stemmer   = 'PorterStemmer'
# Prétraitement
X_sentence_train, tokenizer, sentences_train = preprocess_data_embedding(X_raw=X_train, 
                                                        stem_lem_func=PorterStemmer().stem,
                                                        tokenizer=None, 
                                                        stop_words=stopwords.words('english'), 
                                                        min_count=min_count,
                                                        max_len = max_len, 
                                                        num_words=num_words, 
                                                        return_sentences=True) 
X_sentence_val = preprocess_data_embedding(X_raw=X_val, 
                                                        stem_lem_func=PorterStemmer().stem,
                                                        tokenizer=tokenizer, 
                                                        stop_words=stopwords.words('english'), 
                                                        min_count=1, # mincount = 1 car on est sur le jeu de validation
                                                        max_len = max_len, 
                                                        num_words=num_words) 

def embedding_eval_custom_embed(trial):
    # Hyperparamètres

    ## Embedding
    embedding_model = trial.suggest_categorical('embedding_model',list(embedding_dict.keys()))
    latent_dim = trial.suggest_int('latent_dim', 30, 150)
    window = trial.suggest_int("window", 2, 10)
    sg = trial.suggest_int('sg',0,1)

    ## Modèle
    rnn_size = 64
    ## Entrainement
    epochs = 50
    lr = 1e-3
    ## Savepath des poids du modèle
    model_savepath = "./Models/baselineRNN_pre.h5"



    with mlflow.start_run(nested=True):
        mlflow.log_input(dataset)
        mlflow.log_params(params={
            'num_words':num_words,               
            'max_len': max_len,
            'min_count': min_count,
            'stemmer': stemmer, 
            'latent_dim': latent_dim, 
            'rnn_size': rnn_size, 
            'embedding_model':embedding_model,
            'sg':sg,
            'window':window,
            'epochs': epochs, 
            'learning_rate': lr, 
            "data_size": data_size,
            "data_numrows": data_numrows,
        })


        # Embedding(custom)
        if embedding_model=='Word2Vec':
            embedding_model = Word2Vec(
                sentences=sentences_train, 
                vector_size=latent_dim,  # dimension de l’espace latent
                window=5,         # taille du contexte
                min_count=min_count,      # ignorer les mots trop rares
            workers=4,        # parallélisme CPU
            sg=sg,              # 1 = skip-gram, 0 = CBOW
            epochs=30
            )
        elif embedding_model=='FastText':
            embedding_model = FastText(
                sentences=sentences_train, 
                vector_size=latent_dim, 
                window=5, 
                min_count=min_count,
                workers=4,
                sg=sg,
                epochs=30
                )


        model_vectors = embedding_model.wv
        w2v_words = model_vectors.index_to_key

        embedding_matrix, vocab_size = build_embedding_matrix(tokenizer=tokenizer,
                                          embedding_model=model_vectors, 
                                          latent_dim=latent_dim
                                          )

        
        # Modèle
        model =  build_base_RNN(vocab_size=vocab_size, 
                        latent_dim=latent_dim,
                        input_length=max_len, 
                        embedding_matrix=embedding_matrix,
                        rnn_size = rnn_size)
        ## Callbacks
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]
        ## Compilation
        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        #Entrainement
        with tf.device("/GPU:0"):
            history = model.fit(X_sentence_train, y_train, epochs=epochs, batch_size=64, validation_data=(X_sentence_val,y_val), callbacks=callbacks_list, verbose=0)

        model.load_weights(model_savepath)

        # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(X_sentence_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
        acc = output_dict["Accuracy"]
    return acc

## Definition de l'experiment MLFlow/Optuna



In [30]:
# Création de l'étude Optuna et optimisation
print("Starting hyperparameter optimization with Optuna...")
print("Setting up MLflow experiment...")
mlflow.set_experiment("optuna_word_embedding_experiment_custom_embedding")
exp_id = mlflow.get_experiment_by_name("optuna_word_embedding_experiment_custom_embedding").experiment_id

experiment_description = (
    "Cette experience contient les différents tests pour le modèle RNN simple. "
    "Ici on évalue l'impact du type d'embedding custom et de la dimension de l'espace latent sur un modèle avec simpleRNN"
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "simple-RNN-preprocessing",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Starting hyperparameter optimization with Optuna...
Setting up MLflow experiment...


Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

## Lancement de l'optimisation

In [31]:
# Lancement de l'optimisation avec Optuna
print("Starting optimization trials...")
with mlflow.start_run(run_name="optuna_word_embedding_experiment_custom_embedding"):
    study = optuna.create_study(direction="maximize")
    study.optimize(embedding_eval_custom_embed, n_trials=30)
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_accuracy", study.best_value)

print("Optimization completed.")

[I 2025-10-06 17:41:40,435] A new study created in memory with name: no-name-dd804780-6d92-4aab-bb12-a75613882c2e


Starting optimization trials...
Embedding matrix shape: (7727, 84)
Words found in pretrained embeddings: 7726/7727 (99.99%)
200/200 [==============================] - 2s 9ms/step


2025/10/06 17:44:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/06 17:44:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpwxmun93g\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpwxmun93g\model\data\model\assets
2025/10/06 17:44:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-10-06 17:44:57,006] Trial 0 finished with value: 0.72015625 and parameters: {'embedding_model': 'FastText', 'latent_dim': 84, 'window': 10, 'sg': 1}. Best is trial 0 with value: 0.72015625.


...

Embedding matrix shape: (7727, 85)
Words found in pretrained embeddings: 7726/7727 (99.99%)
200/200 [==============================] - 1s 7ms/step


2025/10/06 19:11:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/06 19:11:30 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpo34oz9_u\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpo34oz9_u\model\data\model\assets
2025/10/06 19:11:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-10-06 19:11:37,400] Trial 29 finished with value: 0.7196875 and parameters: {'embedding_model': 'FastText', 'latent_dim': 85, 'window': 10, 'sg': 1}. Best is trial 7 with value: 0.7278125.


Optimization completed.


In [32]:
client = MlflowClient(tracking_uri="http://localhost:8080")
experiment_id = mlflow.get_experiment_by_name("optuna_word_embedding_experiment_custom_embedding").experiment_id
runs = client.search_runs(experiment_id)

# Métrique pour sélectionner le meilleur modèle
metric_to_optimize = "Accuracy" # liste des métriques enregistrées dans postprocess_data.py ou sur l'UI MLflow
best_run = max(runs, key=lambda run: run.data.metrics.get(metric_to_optimize, float('-inf')))
print(f"Best run ID: {best_run.info.run_id} with metrics:")
for key, value in best_run.data.metrics.items():
    print(f"{key}: {value}")
print(f"Best run parameters:")
for key, value in best_run.data.params.items():
    print(f"{key}: {value}")

# Enregistrement du meilleur modèle
best_model_uri = f"runs:/{best_run.info.run_id}/model"
registered_model_name = "simple_rnn_best_custom_embed"
registered_model = mlflow.register_model(best_model_uri, registered_model_name)
# Enregistrement des paramètres sous forme de tags dans le modèle enregistré
for key, value in best_run.data.params.items():
    print(f"Setting tag {key} = {value} in registered model")
    client.set_model_version_tag(
        name=registered_model_name,
        version=str(registered_model.version),
        key=str(key),
        value=str(value))

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Best run ID: 95c82edbe4b44614969cdb9308b9e4ee with metrics:
Accuracy: 0.7278125
F1_negatif: 0.7193038994521431
F1_positif: 0.7358204428268122
Precision_negatif: 0.7425149700598802
Precision_positif: 0.7147908073070124
Recall_negatif: 0.6975
Recall_positif: 0.758125
ROC_AUC: 0.8050172851562499
Best run parameters:
data_numrows: 32000
data_size: 0.02
embedding_model: Word2Vec
epochs: 50
latent_dim: 103
learning_rate: 0.001
max_len: 30
min_count: 2
num_words: 30000
rnn_size: 64
sg: 1
stemmer: PorterStemmer
window: 2


Registered model 'simple_rnn_best_custom_embed' already exists. Creating a new version of this model...
2025/10/06 19:11:41 WARNING mlflow.tracking._model_registry.fluent: Run with id 95c82edbe4b44614969cdb9308b9e4ee has no artifacts at artifact path 'model', registering model based on models:/m-0132b36ceda745ff9fc51bb6e1c764fa instead
Created version '2' of model 'simple_rnn_best_custom_embed'.


Setting tag data_numrows = 32000 in registered model
Setting tag data_size = 0.02 in registered model
Setting tag embedding_model = Word2Vec in registered model
Setting tag epochs = 50 in registered model
Setting tag latent_dim = 103 in registered model
Setting tag learning_rate = 0.001 in registered model
Setting tag max_len = 30 in registered model
Setting tag min_count = 2 in registered model
Setting tag num_words = 30000 in registered model
Setting tag rnn_size = 64 in registered model
Setting tag sg = 1 in registered model
Setting tag stemmer = PorterStemmer in registered model
Setting tag window = 2 in registered model


# Experimentation sur les embeddings (préentrainés)

Les embeddings pré-entrainés ont été entrainés sur un très grand nombre de tweets et prennent donc en compte un très grand nombre de situations. On peut donc se passer de la phase de stemming et garder les stopwords. 


## Préparation

In [33]:
import gensim.downloader as api
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# Word2Vec Google News
w2v_google = api.load("word2vec-google-news-300")  # KeyedVectors

# FastText wiki-news subwords-300
ft_wiki = api.load("fasttext-wiki-news-subwords-300")

# GloVe Twitter (par exemple 200 dimensions)
glove_tw200 = api.load("glove-twitter-200")
glove_tw100 = api.load("glove-twitter-100")
glove_tw50  = api.load("glove-twitter-50")
glove_tw25  = api.load("glove-twitter-25")



In [34]:
embedding_dict = {'Word2Vec_google':w2v_google, 
                  'FastText_wiki':ft_wiki,
                  'Glove_twitter_200':glove_tw200, 
                  'Glove_twitter_100':glove_tw100, 
                  'Glove_twitter_50':glove_tw50, 
                  'Glove_twitter_25':glove_tw25
                  }

In [35]:
## Prétraitement
min_count = 2
num_words = 30000
max_len   = 30
# Prétraitement
X_sentence_train, tokenizer, sentences_train = preprocess_data_embedding(X_raw=X_train, 
                                                        stem_lem_func=None,
                                                        tokenizer=None, 
                                                        stop_words=None, 
                                                        min_count=min_count,
                                                        max_len = max_len, 
                                                        num_words=num_words, 
                                                        return_sentences=True) 
X_sentence_val = preprocess_data_embedding(X_raw=X_val, 
                                                        stem_lem_func=None,
                                                        tokenizer=tokenizer, 
                                                        stop_words=None, 
                                                        min_count=1, # mincount = 1 car on est sur le jeu de validation
                                                        max_len = max_len, 
                                                        num_words=num_words) 

## Fonction de base 

In [36]:
# Hyperparamètres

## Modèle
rnn_size = 64
## Entrainement
epochs = 50
lr = 1e-3
## Savepath des poids du modèle
model_savepath = "./Models/baselineRNN_pretrained_embed.h5"

In [37]:
def pretrained_embed_experiment(embedding_name):
     with mlflow.start_run():
        mlflow.log_input(dataset)
        latent_dim = embedding_dict[embedding_name].vector_size
        mlflow.log_params(params={
            'num_words':num_words,               
            'max_len': max_len,
            'min_count': min_count,
            'stemmer': 'None', 
            'latent_dim': latent_dim, 
            'rnn_size': rnn_size, 
            'epochs': epochs, 
            'learning_rate': lr,
            'embedding_name':embedding_name,
            "data_size": data_size,
            "data_numrows": data_numrows,
        })


        model_vectors = embedding_dict[embedding_name]

        embedding_matrix, vocab_size = build_embedding_matrix(tokenizer=tokenizer,
                                          embedding_model=model_vectors, 
                                          latent_dim=latent_dim
                                          )

        # Modèle
        model =  build_base_RNN(vocab_size=vocab_size, 
                        latent_dim=latent_dim,
                        input_length=max_len, 
                        embedding_matrix=embedding_matrix,
                        rnn_size = rnn_size)
        ## Callbacks
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]
        ## Compilation
        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        #Entrainement
        with tf.device("/GPU:0"):
            history = model.fit(X_sentence_train, y_train, epochs=epochs, batch_size=64, validation_data=(X_sentence_val,y_val), callbacks=callbacks_list, verbose=0)

        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(X_sentence_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
        

## Définition de l'experiment dans MLFlow

In [38]:
# Création de l'étude Optuna et optimisation
print("Starting hyperparameter optimization with Optuna...")
print("Setting up MLflow experiment...")
mlflow.set_experiment("word_embedding_experiment_pretrained_embedding")
exp_id = mlflow.get_experiment_by_name("word_embedding_experiment_pretrained_embedding").experiment_id

experiment_description = (
    "Cette experience contient les différents tests pour le modèle RNN simple. "
    "Ici on teste plusieurs embeddings préentrainées sur de larges corpora, beaucoup d'attentes par rapport aux embeddings avec Glove entrainés sur des tweets"
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "simple-RNN-pretrained-embeddings",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Starting hyperparameter optimization with Optuna...
Setting up MLflow experiment...


## Lancement de l'experiment

In [39]:
for embedding_name in list(embedding_dict.keys()):
    
    print(f"Running test with {embedding_name}")
    pretrained_embed_experiment(embedding_name)


Running test with Word2Vec_google
Embedding matrix shape: (9616, 300)
Words found in pretrained embeddings: 8930/9616 (92.87%)
200/200 [==============================] - 1s 7ms/step


2025/10/06 19:20:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/06 19:20:54 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp0l012xn1\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp0l012xn1\model\data\model\assets
2025/10/06 19:21:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Running test with FastText_wiki
Embedding matrix shape: (9616, 300)
Words found in pretrained embeddings: 9173/9616 (95.39%)
200/200 [==============================] - 1s 7ms/step


...

INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmppy1fe1ys\model\data\model\assets
2025/10/06 19:30:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Running test with Glove_twitter_25
Embedding matrix shape: (9616, 25)
Words found in pretrained embeddings: 9452/9616 (98.29%)
200/200 [==============================] - 1s 6ms/step


2025/10/06 19:33:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/06 19:33:52 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpv3sbe_od\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpv3sbe_od\model\data\model\assets
2025/10/06 19:34:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Enregistrement du meilleur modèle

In [40]:
client = MlflowClient(tracking_uri="http://localhost:8080")
experiment_id = mlflow.get_experiment_by_name("word_embedding_experiment_pretrained_embedding").experiment_id
runs = client.search_runs(experiment_id)

# Métrique pour sélectionner le meilleur modèle
metric_to_optimize = "Accuracy" # liste des métriques enregistrées dans postprocess_data.py ou sur l'UI MLflow
best_run = max(runs, key=lambda run: run.data.metrics.get(metric_to_optimize, float('-inf')))
print(f"Best run ID: {best_run.info.run_id} with metrics:")
for key, value in best_run.data.metrics.items():
    print(f"{key}: {value}")
print(f"Best run parameters:")
for key, value in best_run.data.params.items():
    print(f"{key}: {value}")

# Enregistrement du meilleur modèle
best_model_uri = f"runs:/{best_run.info.run_id}/model"
registered_model_name = "simple_rnn_best_pre"
registered_model = mlflow.register_model(best_model_uri, registered_model_name)
# Enregistrement des paramètres sous forme de tags dans le modèle enregistré
for key, value in best_run.data.params.items():
    print(f"Setting tag {key} = {value} in registered model")
    client.set_model_version_tag(
        name=registered_model_name,
        version=str(registered_model.version),
        key=str(key),
        value=str(value))

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Best run ID: ce06b66ec4ff48e6a994d2485c63701f with metrics:
Accuracy: 0.795875
F1_negatif: 0.7963459499906467
F1_positif: 0.7954018668170143
Precision_negatif: 0.7945128779395296
Precision_positif: 0.7972497802335803
Recall_negatif: 0.7981875
Recall_positif: 0.7935625
ROC_AUC: 0.874631751953125
Best run parameters:
embedding_name: Glove_twitter_200
epochs: 50
latent_dim: 200
learning_rate: 0.001
max_len: 30
min_count: 2
num_words: 20000
rnn_size: 64
stemmer: None


Registered model 'simple_rnn_best_pre' already exists. Creating a new version of this model...
2025/10/06 19:34:05 WARNING mlflow.tracking._model_registry.fluent: Run with id ce06b66ec4ff48e6a994d2485c63701f has no artifacts at artifact path 'model', registering model based on models:/m-6b8da83843cc4061a7b1edb36b9eb5f4 instead
Created version '4' of model 'simple_rnn_best_pre'.


Setting tag embedding_name = Glove_twitter_200 in registered model
Setting tag epochs = 50 in registered model
Setting tag latent_dim = 200 in registered model
Setting tag learning_rate = 0.001 in registered model
Setting tag max_len = 30 in registered model
Setting tag min_count = 2 in registered model
Setting tag num_words = 20000 in registered model
Setting tag rnn_size = 64 in registered model
Setting tag stemmer = None in registered model


# Comparaison des différentes couches de notre réseau de neurones : SimpleRNN vs GRU vs LSTM

## Préparation 

On reprend les paramètres d'embedding de la meilleure run sur embeddings customs et embeddings préentrainés. 

In [41]:
## Prétraitement
min_count = 2
num_words = 20000
max_len   = 30
# Prétraitement
X_sentence_train, tokenizer, sentences_train = preprocess_data_embedding(X_raw=X_train, 
                                                        stem_lem_func=None,
                                                        tokenizer=None, 
                                                        stop_words=None, 
                                                        min_count=min_count,
                                                        max_len = max_len, 
                                                        num_words=num_words, 
                                                        return_sentences=True) 
X_sentence_val = preprocess_data_embedding(X_raw=X_val, 
                                                        stem_lem_func=None,
                                                        tokenizer=tokenizer, 
                                                        stop_words=None, 
                                                        min_count=1, # mincount = 1 car on est sur le jeu de validation
                                                        max_len = max_len, 
                                                        num_words=num_words) 


model_vectors = glove_tw200
latent_dim = glove_tw200.vector_size

embedding_matrix, vocab_size = build_embedding_matrix(tokenizer=tokenizer,
                                embedding_model=model_vectors, 
                                latent_dim=latent_dim
                              )

Embedding matrix shape: (9616, 200)
Words found in pretrained embeddings: 9452/9616 (98.29%)


In [42]:
from Source.preprocess_data import *
rnn_layer_name_list = ['SimpleRNN','GRU','LSTM']


## Fonction de base

In [43]:
## Modèle
rnn_size = 64
## Entrainement
epochs = 50
lr = 1e-3
## Savepath des poids du modèle

def rnn_layer_experiment(rnn_layer_name):
     with mlflow.start_run():
        mlflow.log_input(dataset)

        mlflow.log_params(params={
            'num_words':num_words,               
            'max_len': max_len,
            'min_count': min_count,
            'stemmer': 'None', 
            'latent_dim': latent_dim, 
            'rnn_size': rnn_size, 
            'epochs': epochs, 
            'learning_rate': lr,
            'embedding_name':'Glove_twitter_200',
            'rnn_layer_name':rnn_layer_name,
            "data_size": data_size,
            "data_numrows": data_numrows,
        })
        model_savepath = "./Models/"+rnn_layer_name+"_model_exp.h5"
        # Modèle
        if rnn_layer_name == 'SimpleRNN':
            model =  build_base_RNN(vocab_size=vocab_size, 
                            latent_dim=latent_dim,
                            input_length=max_len, 
                            embedding_matrix=embedding_matrix,
                            rnn_size = rnn_size)
        elif rnn_layer_name == 'GRU':
            model = build_gru_RNN(vocab_size=vocab_size, 
                            latent_dim=latent_dim,
                            input_length=max_len, 
                            embedding_matrix=embedding_matrix,
                            rnn_size = rnn_size)
        elif rnn_layer_name=='LSTM':
            model = build_lstm_RNN(vocab_size=vocab_size, 
                            latent_dim=latent_dim,
                            input_length=max_len, 
                            embedding_matrix=embedding_matrix,
                            rnn_size = rnn_size)
        ## Callbacks
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]
        ## Compilation
        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        #Entrainement
        with tf.device("/GPU:0"):
            history = model.fit(X_sentence_train, y_train, epochs=epochs, batch_size=64, validation_data=(X_sentence_val,y_val), callbacks=callbacks_list, verbose=0)

        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(X_sentence_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
        

## Definition de l'experiment dans MLFLow

In [44]:
# Création de l'étude Optuna et optimisation
print("Setting up MLflow experiment...")
mlflow.set_experiment("rnn_layer_experiment_pretrained_embedding")
exp_id = mlflow.get_experiment_by_name("rnn_layer_experiment_pretrained_embedding").experiment_id

experiment_description = (
    "Comparaison des impact des types de cellules RNN utilisées : SimpleRNN, GRU et LSTM "
    ""
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "RNN_types-pretrained-embeddings",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Setting up MLflow experiment...


## Lancement de l'expériment

In [45]:
for rnn_layer_name in rnn_layer_name_list:
    
    print(f"Running test with {rnn_layer_name}")
    rnn_layer_experiment(rnn_layer_name)


Running test with SimpleRNN
200/200 [==============================] - 1s 6ms/step


2025/10/06 19:36:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/06 19:36:17 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmphoourp89\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmphoourp89\model\data\model\assets
2025/10/06 19:36:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Running test with GRU
200/200 [==============================] - 1s 3ms/step


...

INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpobygs7eb\model\data\model\assets
2025/10/06 19:38:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Running test with LSTM
200/200 [==============================] - 2s 4ms/step


2025/10/06 19:40:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/06 19:40:02 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpy3apnxxi\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpy3apnxxi\model\data\model\assets
2025/10/06 19:40:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


# Retour sur la longueur des séquences     

Ici on va revenir sur la longueur des séquences utilisées car LSTM permet de garder des séquences plus longues sans pour autant avoir d'évanescence de gradient. 

## Fonction de base 

In [46]:


## Modèle
rnn_size = 64
## Entrainement
epochs = 50
lr = 1e-3
## Savepath des poids du modèle

def lstm_maxlen_experiment(max_len):
     with mlflow.start_run():
        latent_dim = glove_tw200.vector_size
        mlflow.log_input(dataset)
        mlflow.log_params(params={
            'num_words':num_words,               
            'max_len': max_len,
            'min_count': min_count,
            'stemmer': 'None', 
            'latent_dim': latent_dim, 
            'rnn_size': rnn_size, 
            'epochs': epochs, 
            'learning_rate': lr,
            'embedding_name':'Glove_twitter_200',
            'rnn_layer_name':'LSTM',
            "data_size": data_size,
            "data_numrows": data_numrows,
        })

        ## Prétraitement

        # Prétraitement
        X_sentence_train, tokenizer, sentences_train = preprocess_data_embedding(X_raw=X_train, 
                                                        stem_lem_func=None,
                                                        tokenizer=None, 
                                                        stop_words=None, 
                                                        min_count=min_count,
                                                        max_len = max_len, 
                                                        num_words=num_words, 
                                                        return_sentences=True) 
        X_sentence_val = preprocess_data_embedding(X_raw=X_val, 
                                                        stem_lem_func=None,
                                                        tokenizer=tokenizer, 
                                                        stop_words=None, 
                                                        min_count=1, # mincount = 1 car on est sur le jeu de validation
                                                        max_len = max_len, 
                                                        num_words=num_words) 


        model_vectors = glove_tw200

        embedding_matrix, vocab_size = build_embedding_matrix(tokenizer=tokenizer,
                                embedding_model=model_vectors, 
                                latent_dim=latent_dim
                              )
        model_savepath = f"./Models/{rnn_layer_name}_model_exp_len{max_len}.h5"
        # Modèle

        model = build_lstm_RNN(vocab_size=vocab_size, 
                            latent_dim=latent_dim,
                            input_length=max_len, 
                            embedding_matrix=embedding_matrix,
                            rnn_size = rnn_size)
        ## Callbacks
        checkpoint = ModelCheckpoint(model_savepath, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min')
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
        lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=0, min_lr=1e-5)
        callbacks_list = [checkpoint, es, lr_scheduler]
        ## Compilation
        optimizer = Adam(learning_rate=lr)
        model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])

        #Entrainement
        with tf.device("/GPU:0"):
            history = model.fit(X_sentence_train, y_train, epochs=epochs, batch_size=64, validation_data=(X_sentence_val,y_val), callbacks=callbacks_list, verbose=0)

        model.load_weights(model_savepath)

                # Prédictions sur le jeu de validation
        y_pred_proba = model.predict(X_sentence_val)
        y_pred = (y_pred_proba>0.5)


        output_dict = postprocess_model_output(y_val, y_pred, y_pred_proba) # voir postprocess_data.py

        # Logging des métriques dans MLflow
        mlflow.log_metrics(output_dict)
        # Matrice de confusion
        cm = confusion_matrix(y_val, y_pred, normalize='pred')
        fig, ax = plt.subplots()
        sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", ax=ax, )
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix - Validation Set")
        fig.savefig("confusion_matrix.png")
        plt.close(fig)
        mlflow.log_artifact("confusion_matrix.png")
        #
        fig2 = plot_training_history(history,show=False)
        fig2.savefig("learning_path.png")
        plt.close(fig2)
        mlflow.log_artifact("learning_path.png")

        # Enregistrement du modèle dans MLflow
        mlflow.tensorflow.log_model(model, "model")
        

## Experiment MLFLow



In [47]:
# Création de l'étude Optuna et optimisation
print("Setting up MLflow experiment...")
mlflow.set_experiment("lstm_maxlen_experiment")
exp_id = mlflow.get_experiment_by_name("lstm_maxlen_experiment").experiment_id

experiment_description = (
    "Comparaison des impact des types de cellules RNN utilisées : SimpleRNN, GRU et LSTM "
    ""
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "Sentiment analysis modelling",
    "model_type": "LSTM_pretrained_embedding",
    "team": "Ph. Constant",
    "project_quarter": "Q3-2025",
    "mlflow.note.content": experiment_description,
}

for key, value in experiment_tags.items():
    client.set_experiment_tag(exp_id, key, value)

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Setting up MLflow experiment...


## Lancement de l'experiment 

In [48]:
for max_len in list(range(30,100,5)):
    
    print(f"Running test with sequence length of {max_len} tokens")
    lstm_maxlen_experiment(max_len)


Running test with sequence length of 30 tokens
Embedding matrix shape: (9616, 200)
Words found in pretrained embeddings: 9452/9616 (98.29%)
200/200 [==============================] - 2s 3ms/step


2025/10/06 19:42:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/06 19:42:03 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmprbce8r0f\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmprbce8r0f\model\data\model\assets
2025/10/06 19:42:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Running test with sequence length of 35 tokens
Embedding matrix shape: (9616, 200)
Words found in pretrained embeddings: 9452/9616 (98.29%)
200/200 [==============================] - 1s 3ms/step


...

INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmpij4f0m5k\model\data\model\assets
2025/10/06 20:15:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Running test with sequence length of 95 tokens
Embedding matrix shape: (9616, 200)
Words found in pretrained embeddings: 9452/9616 (98.29%)
200/200 [==============================] - 1s 4ms/step


2025/10/06 20:19:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/06 20:19:30 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp1756tp5f\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\bassm\AppData\Local\Temp\tmp1756tp5f\model\data\model\assets
2025/10/06 20:19:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Enregistrement du mailleur modèle

In [49]:
client = MlflowClient(tracking_uri="http://localhost:8080")
experiment_id = mlflow.get_experiment_by_name("lstm_maxlen_experiment").experiment_id
runs = client.search_runs(experiment_id)

# Métrique pour sélectionner le meilleur modèle
metric_to_optimize = "Accuracy" # liste des métriques enregistrées dans postprocess_data.py ou sur l'UI MLflow
best_run = max(runs, key=lambda run: run.data.metrics.get(metric_to_optimize, float('-inf')))
print(f"Best run ID: {best_run.info.run_id} with metrics:")
for key, value in best_run.data.metrics.items():
    print(f"{key}: {value}")
print(f"Best run parameters:")
for key, value in best_run.data.params.items():
    print(f"{key}: {value}")

# Enregistrement du meilleur modèle
best_model_uri = f"runs:/{best_run.info.run_id}/model"
registered_model_name = "lstm_maxlen_best"
registered_model = mlflow.register_model(best_model_uri, registered_model_name)
# Enregistrement des paramètres sous forme de tags dans le modèle enregistré
for key, value in best_run.data.params.items():
    print(f"Setting tag {key} = {value} in registered model")
    client.set_model_version_tag(
        name=registered_model_name,
        version=str(registered_model.version),
        key=str(key),
        value=str(value))

Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 367, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 465, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1635, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\store\tracking\file_store.py", line 1628, in _read_helper
    result = read_yaml(root, file_name)
  File "c:\ProgramData\anaconda3\envs\AI_env_P7_gpu\lib\site-packages\mlflow\utils\yaml_utils.py", line 107, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' d

Best run ID: b63acc4f64914a5fb77ab6b26a974e13 with metrics:
Accuracy: 0.81365625
F1_negatif: 0.811518159117489
F1_positif: 0.8157463770355035
Precision_negatif: 0.8209375199846518
Precision_positif: 0.8066980382570433
Recall_negatif: 0.8023125
Recall_positif: 0.825
ROC_AUC: 0.89485159765625
Best run parameters:
embedding_name: Glove_twitter_200
epochs: 50
latent_dim: 200
learning_rate: 0.001
max_len: 50
min_count: 2
num_words: 20000
rnn_layer_name: LSTM
rnn_size: 64
stemmer: None


Registered model 'lstm_maxlen_best' already exists. Creating a new version of this model...
2025/10/06 20:19:55 WARNING mlflow.tracking._model_registry.fluent: Run with id b63acc4f64914a5fb77ab6b26a974e13 has no artifacts at artifact path 'model', registering model based on models:/m-612a40cdbb1d4774b96429b02b0740f8 instead
Created version '2' of model 'lstm_maxlen_best'.


Setting tag embedding_name = Glove_twitter_200 in registered model
Setting tag epochs = 50 in registered model
Setting tag latent_dim = 200 in registered model
Setting tag learning_rate = 0.001 in registered model
Setting tag max_len = 50 in registered model
Setting tag min_count = 2 in registered model
Setting tag num_words = 20000 in registered model
Setting tag rnn_layer_name = LSTM in registered model
Setting tag rnn_size = 64 in registered model
Setting tag stemmer = None in registered model


Quand on regarde les améliorations obtenues en augmentant la longueur des séquences, il n'est pas réellement pertinent d'augmenter la longueur des séquences au delà de 50 tokens. 